In [50]:
!pip install bs4
!pip install -U selenium
!pip install pandas
!pip install webdriver-manager
!pip install service
!pip install setproctitle
!pip3 install webdriver_manager

In [80]:
from selenium import webdriver
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import os
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

In [117]:
# need to change so that it takes user input (keyword extraction from user resume)
def search_link(job, city, prov):
    url=f'https://www.linkedin.com/jobs/{job}-jobs-{city}-{prov}?trk=homepage-jobseeker_suggested-search&position=1&pageNum=0'
    return url

wd = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wd.get(search_link('data-scientist', 'waterloo', 'ON'))
# instead of doing a link. find element to click and insert search items
num_jobs = wd.find_element(by=By.CSS_SELECTOR, value='h1>span').get_attribute('innerText')
if num_jobs[-1]=='+':
    num_jobs=int(num_jobs.replace(',', '')[:-1])
else:
    num_jobs=int(num_jobs)
print(num_jobs)

i = 2
list_jobs=[]
while i <= (num_jobs/25)+1: 
    job_lists=wd.find_element(by=By.CLASS_NAME, value='jobs-search__results-list')
    jobs=job_lists.find_elements(by=By.TAG_NAME, value='li') # returns a list
    wd.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    i = i + 1
    try:
        xpath='/html/body/main/div/section/button'
        wd.find_element(by=By.XPATH, value=xpath).click()
    except:
        pass



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/soyeonjang/.wdm/drivers/chromedriver/mac64_m1/98.0.4758.102/chromedriver] found in cache


250


In [131]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []
for job in jobs:
    job_id0 = job.get_attribute('data-id')
    job_id.append(job_id0)

    job_title0 = job.find_element(by=By.CSS_SELECTOR, value='h3').get_attribute('innerText')
    # job_title0 = job.find_element_by_css_selector('h3').get_attribute('innerText')
    job_title.append(job_title0)
    
    company_name0 = job.find_element(by=By.CSS_SELECTOR, value='h4').get_attribute('innerText')
    company_name0 = job.find_element_by_css_selector('h4').get_attribute('innerText')
    company_name.append(company_name0)

    location0 = job.find_element(by=By.CSS_SELECTOR, value='[class=\'job-search-card__location\']').get_attribute('innerText')
    # location0 = job.find_element_by_css_selector('[class='job-result-card__location']').get_attribute('innerText')
    location.append(location0)

    # date0=job.find_element(by=By.CSS_SELECTOR, value='[class=\'job-search-card__listdate\']').get_attribute('innerText')
    date0=job.find_element(by=By.CSS_SELECTOR, value='div>div>time').get_attribute('datetime')
    # date0 = job.find_element_by_css_selector('div>div>time').get_attribute('datetime')
    date.append(date0)
    
    job_link0=job.find_element(by=By.CSS_SELECTOR, value='a').get_attribute('href')
    # job_link0 = job.find_element_by_css_selector('a').get_attribute('href') 
    job_link.append(job_link0)

In [ ]:
jd = []
seniority = []
emp_type = []
job_func = []
industries = []for item in range(len(jobs)):
    job_func0=[]
    industries0=[]
    # clicking job to view job details
    job_click_path = f'/html/body/main/div/section[2]/ul/li[{item+1}]/img'
    job_click = job.find_element_by_xpath(job_click_path).click()
    time.sleep(5)

    jd_path = '/html/body/main/section/div[2]/section[2]/div'
    jd0 = job.find_element_by_xpath(jd_path).get_attribute('innerText')
    jd.append(jd0)

    seniority_path = '/html/body/main/section/div[2]/section[2]/ul/li[1]/span'
    seniority0 = job.find_element_by_xpath(seniority_path).get_attribute('innerText')
    seniority.append(seniority0)

    emp_type_path = '/html/body/main/section/div[2]/section[2]/ul/li[2]/span'
    emp_type0 = job.find_element_by_xpath(emp_type_path).get_attribute('innerText')
    emp_type.append(emp_type0)

    job_func_path = '/html/body/main/section/div[2]/section[2]/ul/li[3]/span'
    job_func_elements = job.find_elements_by_xpath(job_func_path)
    for element in job_func_elements:
        job_func0.append(element.get_attribute('innerText'))
        job_func_final = ', '.join(job_func0)
        job_func.append(job_func_final)industries_path = '/html/body/main/section/div[2]/section[2]/ul/li[4]/span'
        industries_elements = job.find_elements_by_xpath(industries_path)
    for element in industries_elements:
        industries0.append(element.get_attribute('innerText'))
        industries_final = ', '.join(industries0)
        industries.append(industries_final)

In [ ]:
job_data = pd.DataFrame({‘ID’: job_id,
'Date': date,
'Company': company_name,
'Title': job_title,
'Location': location,
'Description': jd,
'Level': seniority,
'Type': emp_type,
'Function': job_func,
'Industry': industries,
'Link': job_link
})# cleaning description column

job_data['Description'] = job_data['Description'].str.replace('\n',' ')job_data.to_excel('LinkedIn Job Data_Data Scientist.xlsx', index = False)